In [746]:
"""
скрипт для обработки данных мониторинга трудоустройства
"""


'\nскрипт для обработки данных мониторинга трудоустройства\n'

In [747]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
import copy
import math
import os

warnings.simplefilter(action='ignore', category=UserWarning)

In [748]:
def check_data(cell:str):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0


In [749]:
path_to_files ='data/poo'
path_to_end_folder = 'data/temp'


In [750]:
# создаем словарь верхнего уровня для каждого поо
high_level_dct = {}

In [751]:
for file in os.listdir(path_to_files):
    name_file = file.split('.xlsx')[0]
    print(name_file)
    df = pd.read_excel(f'{path_to_files}/{file}',skiprows=7,dtype=str)
    print(df.shape)
    df.columns = list(map(str,df.columns))
    # Заполняем пока пропуски в 15 ячейке для каждой специальности
    df['06'] = df['06'].fillna('15 ячейка')

    # очищаем от нан и возможнных пустых пробелов
    code_spec =[spec for spec in df['03'].unique() if spec is not np.nan]
    code_spec =[spec for spec in code_spec if spec != ' ']

    # Создаем список для строк
    row_cat = [f'Строка {i}' for i in range(1,16)]
    # Создаем список для колонок
    column_cat = [f'Колонка {i}' for i in range(7,34)]

    # Создаем словарь нижнего уровня содержащий в себе все данные для каждой специальности
    spec_dict = {}
    for row in row_cat:
        spec_dict[row] = {key:0 for key in column_cat}
    # Изменяем последний ключ на строковый поскольку там будут хранится примечания
   # spec_dict['Колонка 33'] = ''
    # Создаем словарь среднего уровня содержащй данные по всем специальностям
    poo_dct = {key:copy.deepcopy(spec_dict) for key in code_spec}

    high_level_dct[name_file] = copy.deepcopy(poo_dct)

    """
    В итоге получается такая структура
    {БРИТ:{13.01.10:{Строка 1:{Колонка 1:0}}},ТСИГХ:{22.01.10:{Строка 1:{Колонка 1:0}}}}

    """

    current_code = 'Ошибка проверьте правильность заполнения кодов специальностей' # чекбокс для проверки заполнения кода специальности

    idx_row = 1 # счетчик обработанных строк

    # Итерируемся по полученному датафрейму через itertuples
    for row in df.itertuples():
        # если счетчик колонок больше 15 то уменьшаем его до единицы
        if idx_row >15:
            idx_row = 1

        #print(row)
        # Проверяем на незаполненные ячейки и ячейки заполненные пробелами
        if (row[3] is not np.nan) and (row[3] != ' '):
            # если значение ячейки отличается от текущего кода специальности то обновляем значение текущего кода
            if row[3] != current_code:
                current_code = row[3]
            
        data_row = row[7:34] # получаем срез с нужными данными     
        for idx_col,value in enumerate(data_row,start=1):
     
            #print(f'Строка {idx_row}')
            #print(f'Колонка {idx_col}')


            if idx_col + 6 == 33:
                # сохраняем примечания в строке
                high_level_dct[name_file][current_code][f'Строка {idx_row}'][f'Колонка {idx_col + 6}'] = str(value)
            else:                
                high_level_dct[name_file][current_code][f'Строка {idx_row}'][f'Колонка {idx_col + 6}'] += check_data(value)
        
        idx_row += 1
                
            


    
    
    
    


Брит - 2022
(255, 34)
ТСИГХ мониторониг
(225, 34)


In [752]:
high_level_dct['Брит - 2022']

{'43.01.06': {'Строка 1': {'Колонка 7': 21,
   'Колонка 8': 8,
   'Колонка 9': 7,
   'Колонка 10': 100,
   'Колонка 11': 0,
   'Колонка 12': 0,
   'Колонка 13': 9,
   'Колонка 14': 3,
   'Колонка 15': 0,
   'Колонка 16': 1,
   'Колонка 17': 0,
   'Колонка 18': 0,
   'Колонка 19': 0,
   'Колонка 20': 0,
   'Колонка 21': 0,
   'Колонка 22': 0,
   'Колонка 23': 0,
   'Колонка 24': 0,
   'Колонка 25': 0,
   'Колонка 26': 0,
   'Колонка 27': 0,
   'Колонка 28': 0,
   'Колонка 29': 0,
   'Колонка 30': 0,
   'Колонка 31': 0,
   'Колонка 32': 0,
   'Колонка 33': 'пример примечания'},
  'Строка 2': {'Колонка 7': 45,
   'Колонка 8': 21,
   'Колонка 9': 0,
   'Колонка 10': 0,
   'Колонка 11': 0,
   'Колонка 12': 0,
   'Колонка 13': 0,
   'Колонка 14': 0,
   'Колонка 15': 0,
   'Колонка 16': 0,
   'Колонка 17': 0,
   'Колонка 18': 0,
   'Колонка 19': 0,
   'Колонка 20': 0,
   'Колонка 21': 0,
   'Колонка 22': 0,
   'Колонка 23': 0,
   'Колонка 24': 0,
   'Колонка 25': 0,
   'Колонка 26': 0,
   'Ко

In [753]:

# получаем уникальные специальности
all_spec_code = set()
for poo,spec in high_level_dct.items():
    for code_spec,data in spec.items():
        all_spec_code.add(code_spec)



In [754]:
itog_df = {key:copy.deepcopy(spec_dict) for key in all_spec_code}

In [755]:
itog_df

{'43.01.09': {'Строка 1': {'Колонка 7': 0,
   'Колонка 8': 0,
   'Колонка 9': 0,
   'Колонка 10': 0,
   'Колонка 11': 0,
   'Колонка 12': 0,
   'Колонка 13': 0,
   'Колонка 14': 0,
   'Колонка 15': 0,
   'Колонка 16': 0,
   'Колонка 17': 0,
   'Колонка 18': 0,
   'Колонка 19': 0,
   'Колонка 20': 0,
   'Колонка 21': 0,
   'Колонка 22': 0,
   'Колонка 23': 0,
   'Колонка 24': 0,
   'Колонка 25': 0,
   'Колонка 26': 0,
   'Колонка 27': 0,
   'Колонка 28': 0,
   'Колонка 29': 0,
   'Колонка 30': 0,
   'Колонка 31': 0,
   'Колонка 32': 0,
   'Колонка 33': 0},
  'Строка 2': {'Колонка 7': 0,
   'Колонка 8': 0,
   'Колонка 9': 0,
   'Колонка 10': 0,
   'Колонка 11': 0,
   'Колонка 12': 0,
   'Колонка 13': 0,
   'Колонка 14': 0,
   'Колонка 15': 0,
   'Колонка 16': 0,
   'Колонка 17': 0,
   'Колонка 18': 0,
   'Колонка 19': 0,
   'Колонка 20': 0,
   'Колонка 21': 0,
   'Колонка 22': 0,
   'Колонка 23': 0,
   'Колонка 24': 0,
   'Колонка 25': 0,
   'Колонка 26': 0,
   'Колонка 27': 0,
   'Колон

In [756]:
# Складываем результаты

for poo,spec in high_level_dct.items():
    for code_spec,data in spec.items():
        print(code_spec)
        for row,col_data in data.items():
            print(row)
            for col,value in col_data.items():
                print(col)
                if col == 'Колонка 33':
                    itog_df[code_spec][row][col] = value
                itog_df[code_spec][row][col] += value

43.01.06
Строка 1
Колонка 7
Колонка 8
Колонка 9
Колонка 10
Колонка 11
Колонка 12
Колонка 13
Колонка 14
Колонка 15
Колонка 16
Колонка 17
Колонка 18
Колонка 19
Колонка 20
Колонка 21
Колонка 22
Колонка 23
Колонка 24
Колонка 25
Колонка 26
Колонка 27
Колонка 28
Колонка 29
Колонка 30
Колонка 31
Колонка 32
Колонка 33
Строка 2
Колонка 7
Колонка 8
Колонка 9
Колонка 10
Колонка 11
Колонка 12
Колонка 13
Колонка 14
Колонка 15
Колонка 16
Колонка 17
Колонка 18
Колонка 19
Колонка 20
Колонка 21
Колонка 22
Колонка 23
Колонка 24
Колонка 25
Колонка 26
Колонка 27
Колонка 28
Колонка 29
Колонка 30
Колонка 31
Колонка 32
Колонка 33
Строка 3
Колонка 7
Колонка 8
Колонка 9
Колонка 10
Колонка 11
Колонка 12
Колонка 13
Колонка 14
Колонка 15
Колонка 16
Колонка 17
Колонка 18
Колонка 19
Колонка 20
Колонка 21
Колонка 22
Колонка 23
Колонка 24
Колонка 25
Колонка 26
Колонка 27
Колонка 28
Колонка 29
Колонка 30
Колонка 31
Колонка 32
Колонка 33
Строка 4
Колонка 7
Колонка 8
Колонка 9
Колонка 10
Колонка 11
Колонка 12
Колонка 13

In [758]:
itog_df['43.01.06']

{'Строка 1': {'Колонка 7': 42,
  'Колонка 8': 16,
  'Колонка 9': 14,
  'Колонка 10': 155,
  'Колонка 11': 0,
  'Колонка 12': 0,
  'Колонка 13': 9,
  'Колонка 14': 3,
  'Колонка 15': 0,
  'Колонка 16': 2,
  'Колонка 17': 0,
  'Колонка 18': 0,
  'Колонка 19': 0,
  'Колонка 20': 0,
  'Колонка 21': 0,
  'Колонка 22': 0,
  'Колонка 23': 0,
  'Колонка 24': 0,
  'Колонка 25': 0,
  'Колонка 26': 0,
  'Колонка 27': 0,
  'Колонка 28': 0,
  'Колонка 29': 0,
  'Колонка 30': 0,
  'Колонка 31': 0,
  'Колонка 32': 0,
  'Колонка 33': 'пример примечанияпример примечания'},
 'Строка 2': {'Колонка 7': 133,
  'Колонка 8': 98,
  'Колонка 9': 0,
  'Колонка 10': 0,
  'Колонка 11': 0,
  'Колонка 12': 0,
  'Колонка 13': 0,
  'Колонка 14': 0,
  'Колонка 15': 0,
  'Колонка 16': 0,
  'Колонка 17': 0,
  'Колонка 18': 0,
  'Колонка 19': 0,
  'Колонка 20': 0,
  'Колонка 21': 0,
  'Колонка 22': 0,
  'Колонка 23': 0,
  'Колонка 24': 0,
  'Колонка 25': 0,
  'Колонка 26': 0,
  'Колонка 27': 0,
  'Колонка 28': 0,
  'Коло